<a href="https://colab.research.google.com/github/Davids048/ELEC-378/blob/main/CNN_%26_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np
import os
from scipy.io import wavfile
from scipy.fft import fft
import scipy

import sys
import importlib
import time
import math

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import SVC
import sklearn

import pickle

import pandas as pd

#Librosa is likely the first feature extraction library you should try out. 

import librosa
import librosa.display
from sklearn.decomposition import PCA

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


### Data import, don't need to do this again unless need to debug, basically because running these code once will save them to a file, next time just run the code in the next section

In [3]:
%cd '/content/drive/MyDrive/Spring 2023/ELEC 378'
%ls
%cd './kaggle_data/data'

/content/drive/MyDrive/Spring 2023/ELEC 378
'CNN & RNN.ipynb'  'Homework 8'/  'HW 6'/                 rf_predictions.csv
 Datas             'Homework 9'/   kaggle_data/
'Homework 7'/       HW-4/         'Random Forest.ipynb'
/content/drive/MyDrive/Spring 2023/ELEC 378/kaggle_data/data


In [4]:
label_conversion = ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
emo_labels = []
onehot_emo_labels = []

audiolist = []
nums = []
calms = 150
clm = 0

# iterate through all file
points = -1
cnt = 0
for file in os.listdir():
  if cnt == points and points != -1:
    print("breaking")
    break
  # Check whether file is in text format or not
  if file.endswith(".wav"):
    if 'sample' not in file:
      # grab the emotion
      class_lbl = label_conversion.index(file[:-7])
      # one-hot encoding
      if class_lbl == 1:
        clm += 1
      if clm <= calms or class_lbl != 1:
        onehot_emo_labels.append([0]*class_lbl+[1]+[0]*(8-class_lbl-1))
        # no one-hot
        emo_labels.append(class_lbl)
    #print(1)
    # sample = 6 so for this case it's 6
    if clm <= calms or class_lbl != 1:
      nums.append(file[:-4]) # goes to emotion + num (removes .wav)
      # fs, audiofile = wavfile.read(file)
      audiofile, sr = librosa.load(file, sr=48000)

      # add audio as-is to array
      audiolist.append(audiofile)
      cnt += 1
if points == -1: points = cnt

KeyboardInterrupt: ignored

In [ ]:
# convert all data to mono: These data are already normalized if its read using librosa.load
new_audiolist = []
for audio in audiolist:
  audio_mono = librosa.to_mono(audio)
  new_audiolist.append(audio_mono)
audiolist = new_audiolist

In [ ]:
# find min len of all audio files
min_length = float('inf')
for audio in audiolist:
  min_length = min(min_length, audio.shape[0])

In [ ]:
# pickel audio, emo_list, and minlenth
Datas = (audiolist, emo_labels, min_length)

import pickle
with open('/content/drive/MyDrive/Spring 2023/ELEC 378/Datas', 'wb') as fp:
  pickle.dump(Datas, fp)

### Load Data (new tool)


In [5]:
with open("/content/drive/MyDrive/Spring 2023/ELEC 378/Datas", "rb") as fp:   # Unpickling
  data_tupel = pickle.load(fp)

In [12]:
audios = data_tupel[0]
emo_labels = data_tupel[1]
min_len = data_tupel[2]

In [1]:
class AudioData:
  def __init__(self, audios, emo_labels, min_len):
    self.audios = audios
    self.emo_labels = emo_labels
    self.min_len = min_len
    self.sr = 48000

  # make feature matrices:
  def make_mfcc_mat(self):
    n_mfcc = 20
    # Initialize lists to store the feature matrices and labels
    mfccs = []
    for audio in self.audios:
      mfcc = librosa.feature.mfcc(y=audio[:self.min_len], hop_length = 512, n_mels=128, n_fft = 2048,sr=self.sr, n_mfcc=n_mfcc)
      # mfcc = np.ravel(mfcc)
      # Append the feature matrices and label to the lists
      mfccs.append(mfcc)
    print(len(mfccs))
    # Concatenate the feature matrices along the time axis to create feature vectors
    mfccs_mat = np.stack(mfccs)
    print(mfccs_mat.shape)
    return mfccs_mat
  
  def make_chroma_mat(self):
    n_chroma = 12
    chromas = []
    for audio in self.audios:
      chroma = librosa.feature.chroma_stft(y=audio[:self.min_len], sr=self.sr, n_chroma=n_chroma)
      chroma = np.ravel(chroma)
      chromas.append(chroma)
    chromas_mat = np.vstack(chromas)
    return chromas_mat
  
  def make_contrast_mat(self):
    n_contrast = 6
    contrasts = []
    for audio in self.audios:
      contrast = librosa.feature.spectral_contrast(y=audio[:self.min_len], sr=self.sr, n_bands=n_contrast)
      contrast = np.ravel(contrast)
      contrasts.append(contrast)
    contrasts_mat = np.vstack(contrasts)
    return contrasts_mat

  def make_rmse_mat(self):
    rmses = []
    for audio in self.audios:
      energy = librosa.feature.rmse(y=audio)
      print(energy.shape)
      energy = np.ravel(energy)
      rmses.append(energy)
    rmse_mat = np.vstack(rmses)
    return rmse_mat



To use this class, first create a class object, then create features 1 by 1 ( recommended) so that later we can combine features as we want. If we want to change params, just change the functions



In [132]:
# make a data object
audio_obj = AudioData(audios, emo_labels,min_len)

In [133]:
# get features
# feature1 = audio_obj.make_mfcc_mat()
# feature2 = audio_obj.make_chroma_mat()
# feature3 = audio_obj.make_contrast_mat()
feature4 = audio_obj.make_rmse_mat()

1125
(1125, 20, 276)


In [130]:
print(feature1.shape)
print(type(feature1))

(22500, 276)
<class 'numpy.ndarray'>


In [134]:
# Stack the feature vectors horizontally to create the final feature matrix
# X = np.hstack([feature1, feature2, feature3])
X=feature1

In [135]:
print(X.shape)
print(feature1.shape)
# print(feature2.shape)
# print(feature3.shape)

(1125, 20, 276)
(1125, 20, 276)


Split dataset

In [136]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, audio_obj.emo_labels, test_size=0.2, random_state=42)

In [137]:
print(y_train)

[4, 7, 5, 4, 5, 6, 2, 4, 7, 5, 1, 3, 0, 4, 0, 0, 7, 7, 6, 5, 2, 2, 2, 2, 2, 0, 1, 6, 0, 7, 2, 3, 2, 4, 3, 0, 4, 2, 7, 3, 6, 7, 7, 3, 2, 1, 2, 1, 1, 1, 6, 2, 1, 0, 3, 1, 5, 3, 7, 3, 4, 7, 2, 4, 6, 0, 6, 7, 4, 7, 3, 1, 4, 1, 6, 6, 6, 2, 6, 3, 3, 1, 4, 7, 1, 3, 4, 4, 6, 3, 0, 2, 3, 5, 4, 7, 2, 4, 6, 7, 5, 0, 7, 1, 1, 2, 2, 7, 3, 3, 7, 6, 4, 7, 0, 5, 1, 3, 6, 0, 3, 2, 7, 3, 4, 2, 2, 3, 4, 2, 3, 6, 0, 2, 5, 6, 1, 1, 5, 0, 0, 2, 2, 4, 6, 7, 3, 1, 1, 3, 1, 0, 4, 0, 7, 7, 5, 4, 3, 4, 0, 2, 7, 6, 1, 1, 6, 0, 2, 4, 4, 1, 3, 0, 2, 4, 2, 4, 1, 0, 4, 5, 1, 6, 4, 2, 0, 3, 4, 0, 4, 5, 2, 6, 3, 1, 3, 4, 7, 7, 5, 4, 2, 7, 2, 4, 7, 1, 7, 4, 2, 0, 0, 5, 7, 1, 0, 4, 1, 6, 0, 4, 0, 0, 3, 2, 0, 3, 5, 2, 2, 4, 7, 7, 7, 6, 6, 6, 0, 6, 6, 0, 7, 4, 0, 0, 3, 6, 7, 0, 1, 2, 4, 7, 2, 7, 1, 1, 2, 4, 5, 4, 4, 1, 4, 1, 5, 4, 4, 2, 3, 4, 6, 4, 0, 0, 7, 0, 3, 3, 3, 2, 0, 3, 7, 5, 7, 6, 1, 3, 2, 4, 1, 0, 1, 4, 6, 2, 1, 1, 2, 4, 2, 7, 4, 4, 6, 4, 2, 7, 2, 0, 4, 1, 6, 2, 5, 1, 4, 6, 7, 4, 1, 7, 2, 2, 6, 2, 5, 6, 1, 1, 1, 

###Create a data loader


In [138]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class AudioDataSet(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y 
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self, index):
    x = self.X[index]
    y = self.y[index]

    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long).unsqueeze(0)
    return x, y

In [139]:
train_dataset = AudioDataSet(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [140]:
val_dataset = AudioDataSet(X_test, y_test)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [183]:
print(train_dataset.X.shape)

(900, 20, 276)


In [179]:
# CNN Model
class CNN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(CNN, self).__init__()

    # Define the layers of the network
    self.conv1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
    self.relu1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
    self.fc1 = nn.Linear(in_features=345, out_features=128)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

  def forward(self, x):
    x = self.conv1(x)
    print(x.shape)
    x = self.relu1(x)
    print(x.shape)
    x = self.pool1(x)
    print(x.shape)
    x = self.conv2(x)
    print(x.shape)

    x = self.relu2(x)
    x = self.pool2(x)
    x = x.view(x.size(0), -1)

    x = self.fc1(x)
    x = self.relu3(x)
    x = self.fc2(x)
    print(x.shape)
    return x


Train the model

In [ ]:
# init the model
cnn_model = CNN(len(audio_obj.audios), 8)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.01)

for epoch in range(5):
  for idx, (features, label) in enumerate(train_dataloader):
    optimizer.zero_grad()
    output = cnn_model(features)
    loss = criterion(output, label.squeeze())
    loss.backward()
    optimizer.step()
  

    

In [ ]:
# eval
with torch.no_grad():
  correct = 0
  total = 0
  for data, target in val_dataloader:
    target = target.squeeze(dim=1)
    # print(target.size())
    output = cnn_model(data)
    _, predicted = torch.max(output.data, 1)
    print(predicted)
    total += target.size(0)
    correct += torch.sum(torch.eq(predicted, target)).item()
    print(torch.sum(torch.eq(predicted, target)).item())
accuracy = 100 * correct / total
print('Accuracy: {:.2f}%'.format(accuracy))